In [1]:

import pandas as pd
import numpy as np
import os, sys, re

# Obtén la ruta absoluta al directorio 'db'
dir_path = os.path.realpath('db')
# Agrega esta ruta a sys.path
sys.path.insert(0, dir_path)

# Ahora intenta importar DatabasePostgres
from database_postgres import DatabasePostgres

In [95]:
def get_file_ddl(file_path='omop-schema/ddl/0_omop_cdm_schema.sql'):
    with open(file_path, 'r') as f:
        ddl = f.read()
    return ddl

def get_files_by_table(table):
    queries_path = f'omop-schema/queries/{table}'
    queries_files = os.listdir(queries_path)
    queries_file_pahts = [os.path.join(queries_path, file) for file in queries_files]
    return queries_file_pahts

def get_content_file(file_path):    
    with open(file_path, 'r') as file:
        content = file.read()
    return content

def get_query_code(content):
    match = re.search('```sql(.*?)```', content, re.DOTALL)
    if match:
        sql_query = match.group(1).strip()
        return sql_query
    else:
        return None
    
def get_queries_by_table(table):
    queries_files = get_files_by_table(table)
    queries = []
    for file in queries_files:
        content = get_content_file(file)
        query = get_query_code(content)
        if query:
            queries.append(query)
    return queries

def get_queries_by_tables(tables= ['person','condition_occurence','drug_exposure','procedure','observation']):
    queries = []
    for table in tables:
        queries.extend(get_queries_by_table(table))
    return queries

def get_content_by_table(table):
    queries_files = get_files_by_table(table)
    content_data = []
    for file in queries_files:
        content = get_content_file(file)
        if content:
            content_data.append(content)
    return content

def get_content_by_tables(tables= ['person','condition_occurence','drug_exposure','procedure','observation']):
    content_data = []
    for table in tables:
        tmp  = get_content_by_table(table)
        content_data.append(tmp)
    content_data_str = ','.join(content_data) 
    return content_data_str

In [96]:
ddl = get_file_ddl()
content = get_content_by_tables()
queries  = get_queries_by_tables()

# Vanna Train

### install library

In [59]:
#!pip3.11 install vanna

### set model

In [1]:
import vanna as vn
from vanna.remote import VannaDefault

# STEP 01: This is a simple example of how to use the Vanna API
api_key = 'xxxx'
vanna_model_name = 'omop_hospital'# Your model name from https://vanna.ai/account/profile
                    
vn = VannaDefault(model=vanna_model_name, api_key=api_key)


### Train model

In [106]:
ddl = get_file_ddl()
content = get_content_by_tables()
queries  = get_queries_by_tables()

# Train the model with the ddl
vn.train

vn.train(ddl=ddl, documentation=content, sql=queries)


Adding documentation....


'1192479-doc'

In [112]:
vn.connect_to_postgres('localhost','postgres','postgres','postgres',5432)

# The information schema query may need some tweaking depending on your database. This is a good starting point.
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS where table_schema = 'cdm'")

# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)
vn.train(plan=plan)

In [119]:
vn.connect_to_postgres('localhost','postgres','postgres','postgres',5432)

# The information schema query may need some tweaking depending on your database. This is a good starting point.
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS where table_schema = 'vocabularies'")

# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)
vn.train(plan=plan)

In [ ]:
# Train with ddls 

#ddl="""create..."""
#vn.add_ddl(ddl=ddl)

In [171]:
# Train with queries

def get_queries(path='omop-schema/queries/condition/C04.md'):
    import re

    with open(path, 'r') as file:
        content = file.read()

    description_match = re.search(r'## Description\n(.+?)\n##', content, re.DOTALL)
    if description_match:
        description = description_match.group(1).strip()

    query_match = re.search(r'```sql\n(.+?)\n```', content, re.DOTALL)
    if query_match:
        query = query_match.group(1).strip()
    return {'description': description, 'query': query.replace('\n', ' ')}



In [176]:
#tables = ['condition','drug','drug_exposure','general','observation','person','procedure']
tables = ['observation','person','procedure']

for table in tables :
    try:
        path=f'omop-schema/queries/{table}'
        files = [path+'/'+item for item in os.listdir(path)]
        for file in files:
            response = vn.add_question_sql(
                question=get_queries(file)['description'],
                sql=get_queries(file)['query']
            )
            print(response)
    except:
        pass

9eefabc5bbe0f68d2e592dd266acb67a-sql
1961c1562866c9ccbb1e57ac246cbc92-sql
73b48891b582d353995521d43b7402ea-sql
0725fd9974603e3629f43de4f09d44d8-sql
a3e3475e11340bf7f129a018b99ca756-sql
aec9f48e8ce4857aed24f11b5dea030e-sql
f81bcbe6a39dbdec95a0225fc8ff400c-sql
996573f11aabf20d54014e7007951859-sql
812e1890341032011da52d4a3b1c03be-sql
52f22020081819c1a65a0e2f1b9e8a93-sql
aec9f48e8ce4857aed24f11b5dea030e-sql
032715ba852e88b89ea6291c8d0b460e-sql


In [105]:
# remove train data
#for id in vn.get_training_data()['id']:
#    vn.remove_training_data(id)

In [177]:
# Training Model with ddl, data and documentation
vn.get_training_data()

,id,training_data_type,question,content
0,349446-sql,sql,Count the genders (gender_concept_id) across a...,"SELECT gender_concept_id, concept_name ..."
1,349452-sql,sql,This query is used to count the day of birth a...,"SELECT month_of_birth, day_of_birth, cou..."
2,349374-sql,sql,This query enables a search of all anatomical ...,SELECT c.concept_id AS anatomical_site...
3,349450-sql,sql,"This query is similar to PE02, but it lists al...","SELECT person.gender_concept_id, concept.c..."
4,1192894-doc,documentation,None,The following columns are in the concept_relat...
...,...,...,...,...
139,349434-sql,sql,This query is used to count all drugs (drug_co...,"SELECT drug_concept_id, condition_concept_..."
140,349417-sql,sql,## Query\nThe following is a sample run of the...,"SELECT c.concept_name AS drug_type, COUNT(*) A..."
141,349424-sql,sql,| This query is used to obtain the gender dist...,"SELECT drug.concept_name AS drug_name, \tdrug_..."
142,1192897-doc,documentation,None,The following columns are in the drug_strength...


In [2]:
# Ask a question for generating the SQL

question  =  """I need a JSON response with "query" containing the SQL code, and "explanation" with the query explanation. The query runs in a cdm schema. 
The question is:"""

question=""" Show the gender concept, gender name, and number of persons available in the hospital."""
response  = vn.generate_sql(question=question)

SQL Prompt: [{'role': 'system', 'content': 'You are a SQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE cdm.note (\n    note_id bigint NOT NULL,\n    person_id bigint NOT NULL,\n    note_event_id bigint,\n    provider_id bigint,\n    visit_occurrence_id bigint,\n    visit_detail_id bigint,\n    note_datetime timestamp without time zone NOT NULL,\n    note_event_field_concept_id integer NOT NULL,\n    note_type_concept_id integer NOT NULL,\n    note_class_concept_id integer NOT NULL,\n    encoding_concept_id integer NOT NULL,\n    language_concept_id integer NOT NULL,\n    note_date date,\n    note_title text,\n    note_text text,\n    note_source_value text,\n    CONSTRAINT chk_note_note_title CHECK ((COALESCE(length(note_title), 0) <= 250))\n);\nCOMMENT ON TABLE cdm.note IS \'The NOTE table captures unstructured informatio

In [179]:
print(response)

This request uses more tokens than gpt-3.5-turbo can handle. Please go to https://vanna.ai/account/profile to upgrade your subscription.


# Mistral 

In [33]:
#!pip3.11 install mistralai

In [4]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import ast

api_key = 'xxxxxxxx'# Your API key from https://mistral.ai/account/profile
model = "mistral-large-latest"
client = MistralClient(api_key=api_key)

In [61]:
def get_mistral_response_entities(question):
    question = f"""I need to extract the patient_number and dates in json format, the date format is YYYY-MM-DD 
    and always the oldest date first and then the most recent: {question}"""
    messages = [
        ChatMessage(role="user", content=question)
    ]
    chat_response = client.chat(
            model=model,
            response_format={"type": "json_object"},
            messages=messages,
        )
    response = chat_response.choices[0].message.content
    return response

def get_mistral_queries(question):
    import aiosql,json
    response = get_mistral_response_entities(question)
    queries = aiosql.from_path("db/queries", "psycopg2")
    response = json.loads(response)
    query = queries.conditions_by_person_id.sql % {'person_id':response['patient_number'],
                                        'start_date':response['dates'][0],
                                        'end_date':response['dates'][1],
                                        }
    return query

def get_mistral_sql_response(question):
    import json
    query = get_mistral_queries(question)
    db = DatabasePostgres()
    response_conditions = db.select(query)
    response_conditions['condition_start_date']=response_conditions['condition_start_date'].astype(str)
    response_conditions['condition_end_date']=response_conditions['condition_end_date'].astype(str)

    response_condition_str = json.dumps((
        response_conditions
        [['condition_start_date','condition_end_date','concept_name']]
        .to_dict(orient='records')
    ))
    return response_condition_str

def get_mistral_easy_respnse(response_condition_str):
    question = f""""
    Can you explain the following diagnoses from this JSON in simple, easy-to-understand language? I need the answer in a descriptive text format
    : {response_condition_str}
    """
    messages_details = [
            ChatMessage(role="user", content=question)
        ]
    chat_response_details = client.chat(
            model=model,
            messages=messages_details,
        )
    response_details = chat_response_details.choices[0].message.content
    return response_details


In [62]:
import json

question ="""Can you provide information about the patient with the number 8099577122802309611 between November 1, 2000, and January 1, 2020?"""
response_condition_str = get_mistral_sql_response(question = question)
response_easy_language = get_mistral_easy_respnse(response_condition_str)
json_string = json.dumps({
    'question': question,
    'response_sql': response_condition_str,
    'response_easy_language': response_easy_language
})
json_string

In [64]:
import json

question ="""Can you provide information about the patient with the number 1481698356816854382 between the first of November, 2015, and the first of January, 2020?"""
response_condition_str = get_mistral_sql_response(question = question)
response_easy_language = get_mistral_easy_respnse(response_condition_str)
json_string = json.dumps({
    'question': question,
    'response_sql': response_condition_str,
    'response_easy_language': response_easy_language
})
json_string



'{"question": "Can you provide information about the patient with the number 1481698356816854382 between the first of November, 2015, and the first of January, 2020?", "response_sql": "[{\\"condition_start_date\\": \\"2016-07-24\\", \\"condition_end_date\\": \\"2016-08-16\\", \\"concept_name\\": \\"Viral sinusitis\\"}, {\\"condition_start_date\\": \\"2016-07-22\\", \\"condition_end_date\\": \\"2018-12-07\\", \\"concept_name\\": \\"Stress\\"}, {\\"condition_start_date\\": \\"2018-03-21\\", \\"condition_end_date\\": \\"2018-04-10\\", \\"concept_name\\": \\"Acute bronchitis\\"}, {\\"condition_start_date\\": \\"2019-12-13\\", \\"condition_end_date\\": \\"None\\", \\"concept_name\\": \\"Retinopathy due to type 2 diabetes mellitus\\"}, {\\"condition_start_date\\": \\"2019-12-13\\", \\"condition_end_date\\": \\"2020-06-12\\", \\"concept_name\\": \\"Stress\\"}]", "response_easy_language": "Sure, I\'d be happy to help explain these medical conditions in simpler terms:\\n\\n1. Viral Sinusitis: T

In [73]:
question ="""I need details about the patient identified by 889381963201066879 for the period from the first of November, 2018, to the thirty-first of December, 2022."""
response_condition_str = get_mistral_sql_response(question = question)
response_easy_language = get_mistral_easy_respnse(response_condition_str)
json_string = json.dumps({
    'question': question,
    'response_sql': response_condition_str,
    'response_easy_language': response_easy_language
})
json_string


'{"question": "I need details about the patient identified by 889381963201066879 for the period from the first of November, 2018, to the thirty-first of December, 2022.", "response_sql": "[{\\"condition_start_date\\": \\"2019-01-08\\", \\"condition_end_date\\": \\"2019-02-05\\", \\"concept_name\\": \\"Stress\\"}, {\\"condition_start_date\\": \\"2019-04-09\\", \\"condition_end_date\\": \\"None\\", \\"concept_name\\": \\"Myocardial infarction\\"}]", "response_easy_language": "Sure, I\'d be happy to help explain these diagnoses to you.\\n\\n1. The first diagnosis is for a condition called \\"Stress\\". This period of stress started on January 8, 2019, and it continued until February 5, 2019. Stress is a feeling of emotional or physical tension. It can come from any event or thought that makes you feel frustrated, angry, or nervous. Stress is the body\'s reaction to a challenge or demand. In short, it\'s your body\'s way of preparing to face a tough situation.\\n\\n2. The second diagnosis 